In [3]:
import pandas as pd
import requests
import time
import os
from datetime import datetime, timedelta

# --- KONFIGURATION ---
NEWS_API_KEY = "41f7159b90304d589a6156bf6f726ab6" 
CSV_DATEIPFAD = "WikiNasdaq_100_constituents.csv" 
OUTPUT_CSV = "gesammelte_nasdaq_news.csv" 
STATUS_FILE = "status.txt" # Speichert den Ticker des zuletzt verarbeiteten Unternehmens

MAX_RETRIES = 3     # Maximale Versuche pro API-Anruf
WAIT_TIME_SECONDS = 2 # Wartezeit zwischen erfolgreichen Abfragen (API-Schonung)
RETRY_WAIT_SECONDS = 10 # Längere Wartezeit bei Serverfehlern

# --- HILFSFUNKTIONEN ---

def load_progress():
    """Lädt den Ticker des zuletzt erfolgreich verarbeiteten Unternehmens."""
    if os.path.exists(STATUS_FILE):
        with open(STATUS_FILE, 'r') as f:
            return f.read().strip()
    return None

def save_progress(ticker):
    """Speichert den Ticker des zuletzt erfolgreich verarbeiteten Unternehmens."""
    with open(STATUS_FILE, 'w') as f:
        f.write(ticker)

# --- KERNFUNKTION ---

def fetch_news_for_company(company_name, ticker, max_retries):
    """Ruft NewsAPI-Headlines für ein Unternehmen ab mit Retry-Logik."""
    
    query = f'"{company_name}" OR "{ticker}"'
    url = "https://newsapi.org/v2/everything"
    
    # Abruf der letzten 7 Tage
    seven_days_ago = (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d')
    
    params = {
        'q': query,
        'language': 'en',
        'sortBy': 'publishedAt',
        'apiKey': NEWS_API_KEY,
        'from': seven_days_ago,
        'pageSize': 100 
    }
    
    for attempt in range(max_retries):
        try:
            print(f"-> Versuch {attempt + 1}/{max_retries}: Suche News für {ticker}...")
            response = requests.get(url, params=params)
            response.raise_for_status() # Löst Fehler für 4xx/5xx Statuscodes aus
            data = response.json()
            
            articles = data.get('articles', [])
            print(f"   -> Gefunden: {len(articles)} Artikel.")
            
            company_news = []
            for article in articles:
                company_news.append({
                    'ticker': ticker,
                    'company_name': company_name,
                    'title': article.get('title'),
                    'description': article.get('description'),
                    'published_at': article.get('publishedAt'),
                    'source_name': article.get('source', {}).get('name'),
                    'url': article.get('url')
                })
                
            return company_news
            
        except requests.exceptions.HTTPError as e:
            status_code = e.response.status_code
            if status_code == 429:
                print("!!! LIMIT ERREICHT (429 Too Many Requests). Skript stoppt und speichert Fortschritt.")
                raise # Stoppt die Hauptschleife
            elif status_code in [500, 502, 503, 504]:
                # Temporäre Serverfehler
                print(f"!!! Temporärer Serverfehler ({status_code}). Warte {RETRY_WAIT_SECONDS}s und versuche erneut.")
                time.sleep(RETRY_WAIT_SECONDS)
                continue
            else:
                # Andere Fehler (z.B. 401, 404)
                print(f"!!! Dauerhafter HTTP Fehler ({status_code}) bei {ticker}. Wird übersprungen.")
                return []
                
        except requests.exceptions.RequestException as e:
            # Fehler wie ConnectionError (z.B. Ausfall)
            print(f"!!! Verbindungsfehler bei {ticker}: {e}. Warte {RETRY_WAIT_SECONDS}s und versuche erneut.")
            time.sleep(RETRY_WAIT_SECONDS)
            continue
            
    print(f"!!! ALLE VERSUCHE FÜR {ticker} SIND GESCHEITERT. Wird übersprungen.")
    return []

# --- HAUPTPROGRAMM ---

def main():
    """Hauptfunktion zur Koordination des Crawlings."""
    
    try:
        df_nasdaq = pd.read_csv(CSV_DATEIPFAD)
        # KORREKTUR: Prüfung auf 'Company' und 'Ticker'
        if 'Ticker' not in df_nasdaq.columns or 'Company' not in df_nasdaq.columns:
            raise ValueError(f"CSV muss die Spalten 'Ticker' und 'Company' enthalten. Gefundene Spalten: {df_nasdaq.columns.tolist()}")
    except Exception as e:
        print(f"Fehler beim Laden der CSV-Datei: {e}")
        return

    # 1. Fortschritt laden
    last_processed_ticker = load_progress()
    start_crawling = True if last_processed_ticker is None else False
    
    # Laden alter Daten, falls vorhanden
    if os.path.exists(OUTPUT_CSV):
        df_existing = pd.read_csv(OUTPUT_CSV)
        all_articles = df_existing.to_dict('records')
        print(f"Lade {len(all_articles)} existierende Artikel aus {OUTPUT_CSV}.")
    else:
        all_articles = []

    
    print(f"\nStarte Crawling. Letzter Ticker war: {last_processed_ticker if last_processed_ticker else 'Keiner'}")

    # 2. Schleife durch alle Unternehmen
    for index, row in df_nasdaq.iterrows():
        ticker = row['Ticker']
        
        # Springen zu dem Ticker, wo wir aufgehört haben
        if not start_crawling:
            if ticker == last_processed_ticker:
                start_crawling = True
                print(f"Fortsetzung gefunden: Starte nach {ticker}.")
            continue
        
        company_name = row['Company']
        
        # 3. News abrufen
        try:
            news_data = fetch_news_for_company(company_name, ticker, MAX_RETRIES)
            all_articles.extend(news_data)
            
            # 4. Fortschritt speichern und warten
            save_progress(ticker) # Speichert den Ticker nach erfolgreicher Verarbeitung
            time.sleep(WAIT_TIME_SECONDS) 
            
        except Exception:
            # Wird bei 429-Fehler ausgelöst, bricht die Schleife ab, Fortschritt ist gespeichert
            print("Crawling beendet wegen API-Limit.")
            break 
            

    # 5. Daten konsolidieren und speichern
    if all_articles:
        df_results = pd.DataFrame(all_articles)
        
        # Deduplizierung basierend auf der URL und Ticker
        initial_count = len(df_results)
        df_results.drop_duplicates(subset=['url', 'ticker'], inplace=True) 
        final_count = len(df_results)
        
        print("\n--- ZUSAMMENFASSUNG ---")
        print(f"Gesamtanzahl Artikel (nach Deduplizierung): {final_count}")
        print(f"Anzahl entfernter Duplikate: {initial_count - final_count}")
        
        df_results.to_csv(OUTPUT_CSV, index=False)
        print(f"Daten erfolgreich gespeichert in: {OUTPUT_CSV}")
    
    # Aufräumen des Status-Files, wenn alle Unternehmen durchlaufen wurden
    if start_crawling and (index + 1) == len(df_nasdaq):
        os.remove(STATUS_FILE)
        print("Alle Unternehmen verarbeitet. Statusdatei entfernt.")


if __name__ == "__main__":
    main()


Starte Crawling. Letzter Ticker war: Keiner
-> Versuch 1/3: Suche News für ADBE...
!!! Temporärer Serverfehler (500). Warte 10s und versuche erneut.
-> Versuch 2/3: Suche News für ADBE...
   -> Gefunden: 4 Artikel.
-> Versuch 1/3: Suche News für AMD...
   -> Gefunden: 99 Artikel.
-> Versuch 1/3: Suche News für ABNB...
   -> Gefunden: 96 Artikel.
-> Versuch 1/3: Suche News für GOOGL...
!!! Temporärer Serverfehler (500). Warte 10s und versuche erneut.
-> Versuch 2/3: Suche News für GOOGL...
!!! Temporärer Serverfehler (500). Warte 10s und versuche erneut.
-> Versuch 3/3: Suche News für GOOGL...
   -> Gefunden: 56 Artikel.
-> Versuch 1/3: Suche News für GOOG...
   -> Gefunden: 31 Artikel.
-> Versuch 1/3: Suche News für AMZN...
   -> Gefunden: 99 Artikel.
-> Versuch 1/3: Suche News für AEP...
   -> Gefunden: 5 Artikel.
-> Versuch 1/3: Suche News für AMGN...
   -> Gefunden: 46 Artikel.
-> Versuch 1/3: Suche News für ADI...
   -> Gefunden: 50 Artikel.
-> Versuch 1/3: Suche News für AAPL...


In [8]:
import pandas as pd
import requests
import time
from datetime import datetime
import os
import sys

# --- 1. KONFIGURATION ---
# WICHTIG: Ersetzen Sie die Platzhalter durch Ihre tatsächlichen NewsAPI-Keys
API_KEYS = [
    "41f7159b90304d589a6156bf6f726ab6", 
    "a80f40f0df954696b206f77b1e91efd9" 
]

# Dateipfade
CSV_DATEIPFAD = "WikiNasdaq_100_constituents.csv" 
OUTPUT_CSV = "gesammelte_nasdaq_news_doublekey.csv"
PROGRESS_FILE = "crawling_progress.txt" # Speichert den Ticker des letzten erfolgreich verarbeiteten Unternehmens

# Die korrekten Spaltennamen aus Ihrer CSV-Datei
TICKER_COLUMN = 'Ticker'
NAME_COLUMN = 'Company'


# --- 2. HILFSFUNKTIONEN ---

def fetch_news_for_company(company_name, ticker, api_key):
    """Ruft NewsAPI-Headlines für ein Unternehmen mit einem spezifischen Key ab."""
    
    query = f'"{company_name}" OR "{ticker}"'
    url = "https://newsapi.org/v2/everything"
    
    # Abfrage der letzten 7 Tage
    seven_days_ago = (datetime.now() - pd.Timedelta(days=7)).strftime('%Y-%m-%d')
    
    params = {
        'q': query,
        'language': 'en',
        'sortBy': 'publishedAt',
        'apiKey': api_key,
        'from': seven_days_ago,
        'pageSize': 100 
    }
    
    try:
        print(f"  -> Suche News für: {ticker} ({company_name})...")
        response = requests.get(url, params=params)
        response.raise_for_status() 
        data = response.json()
        
        articles = data.get('articles', [])
        print(f"  -> Gefunden: {len(articles)} Artikel.")
        
        company_news = []
        for article in articles:
            company_news.append({
                'ticker': ticker,
                'company_name': company_name,
                'title': article.get('title'),
                'description': article.get('description'),
                'published_at': article.get('publishedAt'),
                'source_name': article.get('source', {}).get('name'),
                'url': article.get('url')
            })
            
        return company_news
        
    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 429:
            # Spezifische Behandlung des Limit-Fehlers (429)
            print(f"!!! LIMIT ERREICHT (429) für {api_key[-4:]}. Abbruch dieses Teils.")
            # Wirft einen spezifischen Fehler, der in process_part abgefangen wird
            raise Exception("API_LIMIT_REACHED") 
        
        print(f"!!! HTTP Fehler bei {ticker} ({api_key[-4:]}): {e.response.status_code}. Wird übersprungen.")
        return []
    
    except Exception as e:
        print(f"!!! Allgemeiner Fehler bei {ticker}: {e}. Wird übersprungen.")
        return []

def save_progress(ticker):
    """Speichert den Ticker des zuletzt erfolgreich verarbeiteten Unternehmens."""
    with open(PROGRESS_FILE, 'w') as f:
        f.write(ticker)

def load_progress():
    """Lädt den Ticker, ab dem das Crawling fortgesetzt werden soll."""
    if os.path.exists(PROGRESS_FILE):
        with open(PROGRESS_FILE, 'r') as f:
            return f.read().strip()
    return None

def process_part(df_part, api_key_index, all_articles):
    """Verarbeitet einen DataFrame-Teil mit einem spezifischen API-Key."""
    api_key = API_KEYS[api_key_index]
    
    # Prüfen, ob bereits Fortschritt aus früheren Läufen gespeichert wurde
    last_processed_ticker = load_progress()
    start_crawling = True if last_processed_ticker is None else False
    
    key_suffix = api_key[-4:]

    print(f"\n--- STARTE TEIL {api_key_index + 1} MIT KEY ***{key_suffix} ---")
    
    for index, row in df_part.iterrows():
        # Lesen der Daten mit den konfigurierten Spaltennamen
        ticker = row[TICKER_COLUMN]
        company_name = row[NAME_COLUMN]
        
        # Logik zum Wiederaufsetzen nach einem Absturz/Limit
        if not start_crawling:
            if ticker == last_processed_ticker:
                # Das letzte verarbeitete Element wurde gefunden, mit dem nächsten fortsetzen.
                start_crawling = True
                print(f"*** Fortschritt geladen. Starte ab dem nächsten Unternehmen nach {last_processed_ticker}.")
            continue
        
        # Daten abrufen
        try:
            news_data = fetch_news_for_company(company_name, ticker, api_key)
            all_articles.extend(news_data)
            
            # Nur bei Erfolg den Fortschritt speichern
            save_progress(ticker)
            
        except Exception as e:
            if str(e) == "API_LIMIT_REACHED":
                print(f"\n!!! API-Limit für Key ***{key_suffix} erreicht. Beende diesen Teil.")
                return False # Signalisiert, dass das Limit erreicht wurde
            raise # Wirft andere Fehler weiter
            
        # Wichtig: Zeitverzögerung zwischen Anfragen (reduziert die Wahrscheinlichkeit eines 429)
        time.sleep(1.5) 
        
    return True # Signalisiert, dass der Teil erfolgreich abgeschlossen wurde

# --- 3. HAUPTPROGRAMM ---

def main():
    
    # 1. CSV einlesen und Validierung
    try:
        df_nasdaq = pd.read_csv(CSV_DATEIPFAD)
        if len(API_KEYS) < 2:
             print("FEHLER: Bitte beide API-Keys im Skript konfigurieren.")
             sys.exit(1)
        
        if TICKER_COLUMN not in df_nasdaq.columns or NAME_COLUMN not in df_nasdaq.columns:
             print(f"FEHLER: Die CSV-Datei muss die Spalten '{TICKER_COLUMN}' und '{NAME_COLUMN}' enthalten. Bitte in der Konfiguration prüfen.")
             sys.exit(1)
             
    except FileNotFoundError:
        print(f"FEHLER: Die Datei '{CSV_DATEIPFAD}' wurde nicht gefunden.")
        sys.exit(1)
    except Exception as e:
        print(f"Fehler beim Laden der CSV-Datei: {e}")
        sys.exit(1)

    total_companies = len(df_nasdaq)
    split_point = total_companies // 2 
    
    # 2. DataFrame aufteilen
    df_parts = [df_nasdaq.iloc[:split_point], df_nasdaq.iloc[split_point:]]
    all_articles = []
    
    # 3. Teile nacheinander mit den dedizierten Keys verarbeiten
    for i in range(len(API_KEYS)):
        df_part = df_parts[i]
        
        # Starte die Verarbeitung des Teils. 
        success = process_part(df_part, i, all_articles)
        
        # Wenn das Limit erreicht wurde, versuchen wir den nächsten Key (falls vorhanden).
        if not success:
            continue
    
    # 4. Daten konsolidieren und speichern
    if all_articles:
        df_results = pd.DataFrame(all_articles)
        
        # Deduplizierung basierend auf der URL 
        df_results.drop_duplicates(subset=['url'], inplace=True) 
        
        print("\n--- ZUSAMMENFASSUNG ---")
        print(f"Gesamtanzahl Artikel (nach Deduplizierung): {len(df_results)}")
        
        # Speichern im "append" Modus, um Fortschritte zu sichern
        # 'header=...' stellt sicher, dass der Header nur einmal geschrieben wird
        write_header = not os.path.exists(OUTPUT_CSV) or os.stat(OUTPUT_CSV).st_size == 0
        df_results.to_csv(OUTPUT_CSV, index=False, mode='a', header=write_header)
        print(f"Daten erfolgreich angehängt/gespeichert in: {OUTPUT_CSV}")
        
        # Beim erfolgreichen Abschluss des gesamten Crawlings die Fortschrittsdatei löschen
        if os.path.exists(PROGRESS_FILE):
             os.remove(PROGRESS_FILE)
             print("Crawling abgeschlossen. Fortschrittsdatei entfernt.")
    else:
        print("\nKeine neuen Artikel gefunden oder Fehler aufgetreten.")

if __name__ == "__main__":
    main()


--- STARTE TEIL 1 MIT KEY ***6ab6 ---
  -> Suche News für: ADBE (Adobe Inc.)...
!!! LIMIT ERREICHT (429) für 6ab6. Abbruch dieses Teils.

!!! API-Limit für Key ***6ab6 erreicht. Beende diesen Teil.

--- STARTE TEIL 2 MIT KEY ***efd9 ---
  -> Suche News für: INTU (Intuit)...
  -> Gefunden: 96 Artikel.
  -> Suche News für: ISRG (Intuitive Surgical)...
  -> Gefunden: 12 Artikel.
  -> Suche News für: KDP (Keurig Dr Pepper)...
  -> Gefunden: 15 Artikel.
  -> Suche News für: KLAC (KLA Corporation)...
  -> Gefunden: 8 Artikel.
  -> Suche News für: KHC (Kraft Heinz)...
  -> Gefunden: 16 Artikel.
  -> Suche News für: LRCX (Lam Research)...
  -> Gefunden: 25 Artikel.
  -> Suche News für: LIN (Linde plc)...
  -> Gefunden: 95 Artikel.
  -> Suche News für: LULU (Lululemon)...
  -> Gefunden: 92 Artikel.
  -> Suche News für: MAR (Marriott International)...
  -> Gefunden: 92 Artikel.
  -> Suche News für: MRVL (Marvell Technology)...
  -> Gefunden: 15 Artikel.
  -> Suche News für: MELI (Mercado Libre)